**Tutorial**

In this tutorial we will process NDVI satellite imagery and socioeconomic census data. Merging satellite imagery (NDVI vegetation index) with socioeconomic census data is crucial for regional agricultural planning because it allows decision-makers to connect environmental conditions with human factors. 

**NDVI (Normalized Difference Vegetation Index)** shows the health and density of vegetation based on satellite imagery.

**Census data** provides insights into population, income levels, employment, and food security.

Why merge them? 
- If a region has declining vegetation health (low NDVI) and high dependency on agriculture, it may indicate a risk of economic downturn or food shortages.
- This allows governments and organizations to prioritize funding, subsidies, and disaster relief where it’s needed most.
- If a region has both low NDVI and high food insecurity, early interventions (like food aid, better infrastructure, or market access improvements) can prevent a humanitarian crisis.
- This ensures sustainable land use planning by balancing economic needs with environmental conservation.
- This helps plan better roads, storage facilities, and transportation networks to ensure farmers can get their goods to market efficiently.

We will do this study for the eastern side of Belgium. 
1. Load NDVI Data: Reads satellite imagery as a raster and extracts NDVI values from eastern Belgium. [source](https://viewer.terrascope.be/?language=en&bbox=1.5633742078460393,49.69944636941702,8.363743279358971,51.590223470510296&overlay=true&bgLayer=OSM&date=2025-03-24&layer=terrascope-s2-ndvi-v2_ndvi)
2. Load Census Data: Reads socioeconomic data from a geojson. We will focus on population and employment indices. It's clipped data from the NDVI [source](https://ec.europa.eu/eurostat/web/gisco/geodata/population-distribution/geostat).
3. Load Density Population: Reads a raster dataset with estimated population density per grid-cell for all Belgium [source](https://hub.worldpop.org/geodata/summary?id=45254).
4. Convert Locations to H3 DGGS: Assigns each census data point and NDVI pixel to an H3 hexagonal index.
5. Aggregate both raster to H3 Cells: Computes the average NDVI per hex cell.

<h2>Step1 - Install libraries and load and convert NDVI to DGGS format</h2>

**Installing libraries**

In [ ]:
pip install rasterio geopandas pandas shapely

In [ ]:
pip install h3 pydeck

**Importing libraries**

In [ ]:
import rasterio
import geopandas as gpd
import h3
import pandas as pd
from shapely.geometry import Point

**Declare variables and datasets (editable)**

In [ ]:
ndvi = "ndvi.tif"
census = "census_east_bel.geojson"
pop_dens = "pop_density_bel.tif" 
resolution = 7

**Declaring functions**

In [ ]:
# Load socioeconomic data (shapefile)
def load_population_data(shapefile):
    return gpd.read_file(shapefile)

# Assign H3 index to population data
def assign_h3_to_population(population_data, resolution=6):
    population_data['h3_index'] = population_data.geometry.apply(lambda geom: h3.latlng_to_cell(geom.y, geom.x, resolution))


    return population_data

# Aggregate raster data into H3 cells
def aggregate(raster, resolution=6, field="value"):
    with rasterio.open(raster) as src:
        array = src.read(1)
        transform = src.transform

        values = []
        for row in range(array.shape[0]):
            for col in range(array.shape[1]):
                lon, lat = transform * (col, row)
                h3_index = h3.latlng_to_cell(lat, lon, resolution)
                values.append((h3_index, float(array[row, col])))

        df = pd.DataFrame(values, columns=['h3_index', field])
        return df.groupby('h3_index', as_index=False).mean()


**Aggregate NDVI data and convert to DGGS format (this is the step that takes longer)**

In [ ]:
ndvi_df = aggregate(ndvi, resolution=resolution, field="ndvi")
ndvi_df["ndvi"] = (ndvi_df["ndvi"] - ndvi_df["ndvi"].min()) / (ndvi_df["ndvi"].max() - ndvi_df["ndvi"].min())

print(ndvi_df)

<h2>Step2 - Load census data and convert to DGGS</h2>

**Load census data**

In [ ]:
population_data = load_population_data(census)

**Convert census data to DGGS format**

In [ ]:
population_data = assign_h3_to_population(population_data, resolution=resolution)
population_data = population_data[["h3_index", "T", "M", "F", "EMP"]]
population_data = population_data.groupby("h3_index", as_index=False).sum()

population_data["EMP_PERC"] = population_data["EMP"] * 100/population_data["T"]
        
print(population_data)

**Aggregate population density data and convert to DGGS format**

In [ ]:
bel_pop_density = aggregate(pop_dens, resolution=resolution, field="population_density")
print(bel_pop_density)

**Merge all datasets**

In [ ]:
df = population_data.merge(ndvi_df, on='h3_index', how='left').dropna()
print(population_data)
df = df.merge(bel_pop_density, on='h3_index', how='left').dropna()
print(df)

<h2>Step3 -Visualize it in the map</h2>

**Visualize DGGS in a map**

In [ ]:

import pydeck as pdk
import pandas as pd


# Define a layer to display on a map
layer = pdk.Layer(
    "H3HexagonLayer",
    df,
    pickable=True,
    stroked=True,
    filled=True,
    extruded=True,
    get_hexagon="h3_index",
    get_fill_color="[0, (1.3-ndvi)  * 255, 0]",
    get_line_color=[255, 255, 255],
    line_width_min_pixels=2,
)

# Set the viewport location
view_state = pdk.ViewState(latitude=50.5010789 , longitude=4.4764595, zoom=6, bearing=0)


# Render
r = pdk.Deck(layers=[layer], initial_view_state=view_state,
             map_style="light",
    tooltip={"text": "NDVI: {ndvi} \n Population: {T} \n Male Population: {M} \n Female population: {F} \n Employed population: {EMP_PERC}%"})
r.show()